## Question 3 ##

**(i)**

In [ ]:
import os
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from skimage.color import rgb2hed
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import pearsonr, spearmanr

# Load dataset
CSV_PATH = "/Users/soumya/Downloads/protein_expression_data.csv"  # CSV file containing image filenames & CD11b expression
IMAGE_FOLDER = "/Users/soumya/Desktop/patches_256/"  # Adjust if necessary

df = pd.read_csv(CSV_PATH)

def format_image_filename(filename, id_value):
    specimen = filename.split('-')[-1]  # Extract last part after hyphen
    formatted_filename = f"{specimen}_{id_value}.png"
    return os.path.join(IMAGE_FOLDER, formatted_filename)

df["image_path"] = df.apply(lambda row: format_image_filename(row["VisSpot"], row["id"]), axis=1)

# Normalize CD11b values
mean_cd11b = df["CD11b"].mean()
std_cd11b = df["CD11b"].std()
df["CD11b"] = (df["CD11b"] - mean_cd11b) / std_cd11b

# Split dataset (Train: B1, C1, D1 | Test: A1)
df["Specimen"] = df["VisSpot"].apply(lambda x: x.split('-')[-1])
test_set = df[df["Specimen"] == "A1"].reset_index(drop=True)
train_set = df[df["Specimen"].isin(["B1", "C1", "D1"])].reset_index(drop=True)

# Image Transformations with Data Augmentation
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
])

# Custom Dataset Class
class ProteinDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        img_path = self.df.iloc[idx]["image_path"]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            image = self.transform(image)
        label = torch.tensor(self.df.iloc[idx]["CD11b"], dtype=torch.float32)
        return image, label

# Creating Data Loaders
train_dataset = ProteinDataset(train_set, transform=transform)
test_dataset = ProteinDataset(test_set, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Define Neural Network Model
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.base_model = models.resnet18(pretrained=True)
        for param in list(self.base_model.parameters())[:-10]:  # Freeze all but last 10 layers initially
            param.requires_grad = False
        self.base_model.fc = nn.Sequential(
            nn.Linear(self.base_model.fc.in_features, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 1),
            nn.Tanh()  # Ensure output is in the normalized range
        )
    
    def forward(self, x):
        return self.base_model(x).squeeze()

# Initialize Model
model = CNNModel()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.00005)  # Lower learning rate for stable training
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.0001, steps_per_epoch=len(train_loader), epochs=20)

# Training Loop
num_epochs = 20
train_losses = []
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()
    
        epoch_loss += loss.item()
    train_losses.append(epoch_loss / len(train_loader))
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss / len(train_loader):.4f}")

# Evaluate Model
model.eval()
predictions, true_values = [], []
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        predictions.extend(outputs.numpy())
        true_values.extend(labels.numpy())

# Compute Performance Metrics
rmse = np.sqrt(mean_squared_error(true_values, predictions))
pearson_corr, _ = pearsonr(true_values, predictions)
spearman_corr, _ = spearmanr(true_values, predictions)
r2 = r2_score(true_values, predictions)

print(f"RMSE: {rmse:.4f}")
print(f"Pearson Correlation: {pearson_corr:.4f}")
print(f"Spearman Correlation: {spearman_corr:.4f}")
print(f"R2 Score: {r2:.4f}")

# Plot Training Loss
plt.plot(train_losses, marker='o')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training Loss Over Epochs")
plt.grid()
plt.show()

# Scatter Plot: True vs Predicted
plt.scatter(true_values, predictions, alpha=0.5)
plt.xlabel("True CD11b Expression")
plt.ylabel("Predicted CD11b Expression")
plt.title("True vs Predicted CD11b Expression")
plt.grid()
plt.show()

/opt/homebrew/Caskroom/miniconda/base/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
import os
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from skimage.color import rgb2hed
from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import pearsonr, spearmanr

# Load dataset
CSV_PATH = "/Users/soumya/Downloads/protein_expression_data.csv"
IMAGE_FOLDER = "/Users/soumya/Desktop/patches_256/"

df = pd.read_csv(CSV_PATH)

# Display dataset columns to debug
print("Dataset Columns:", df.columns)
print(df.head())

# Ensure 'VisSpot' column exists
if "VisSpot" not in df.columns:
    raise ValueError("Column 'VisSpot' is missing in the dataset. Check CSV structure.")

# Extract specimen correctly
df["Specimen"] = df["VisSpot"].apply(lambda x: x.split('-')[-1] if '-' in x else "Unknown")

# Keep only valid specimens ('A1', 'B1', 'C1', 'D1')
df = df[df["Specimen"].isin(["A1", "B1", "C1", "D1"])].reset_index(drop=True)

# Display unique specimens and row count
print("Unique Specimens Found:", df["Specimen"].unique())
print(f"Number of rows after filtering: {len(df)}")

# Ensure protein expression columns are numeric
protein_columns = df.columns.difference(["Unnamed: 0", "VisSpot", "Specimen", "id", "Location_Center_Y", "Location_Center_X"])
for col in protein_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Drop NaN values
df = df.dropna(subset=protein_columns).reset_index(drop=True)

# Normalize protein expression values (Min-Max Scaling)
for protein in protein_columns:
    min_val, max_val = df[protein].min(), df[protein].max()
    df[protein] = (df[protein] - min_val) / (max_val - min_val) if min_val != max_val else 0

# Check unique specimens count for GroupKFold
num_unique_specimens = df["Specimen"].nunique()
print(f"Number of Unique Specimens: {num_unique_specimens}")
if num_unique_specimens < 2:
    raise ValueError(f"Not enough unique specimens for GroupKFold. Found {num_unique_specimens} specimens.")

# Generate correct image filenames
def format_image_filename(filename, id_value):
    specimen = filename.split('-')[-1]
    return os.path.join(IMAGE_FOLDER, f"{specimen}_{id_value}.png")

df["image_path"] = df.apply(lambda row: format_image_filename(row["VisSpot"], row["id"]), axis=1)

# Image Transformations
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Custom Dataset
class ProteinDataset(Dataset):
    def __init__(self, df, protein_columns, transform=None):
        self.df = df
        self.protein_columns = protein_columns
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        img_path = self.df.iloc[idx]["image_path"]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            image = self.transform(image)

        # Ensure correct label format
        labels = self.df.iloc[idx][self.protein_columns].values.astype(np.float32)
        labels = torch.tensor(labels, dtype=torch.float32)
        return image, labels

# Define CNN Model
class CNNModel(nn.Module):
    def __init__(self, num_outputs):
        super(CNNModel, self).__init__()
        self.base_model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
        for param in self.base_model.parameters():
            param.requires_grad = False  # Freeze feature extractor
        self.base_model.fc = nn.Sequential(
            nn.Linear(self.base_model.fc.in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_outputs)
        )
    
    def forward(self, x):
        return self.base_model(x)

# Cross-validation with GroupKFold
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gkf = GroupKFold(n_splits=4)
groups = df["Specimen"]

metrics_results = {protein: {"RMSE": [], "Pearson": [], "Spearman": [], "R2": []} for protein in protein_columns}

for train_idx, test_idx in gkf.split(df, groups=groups):
    train_df, test_df = df.iloc[train_idx], df.iloc[test_idx]
    
    train_dataset = ProteinDataset(train_df, protein_columns, transform=transform)
    test_dataset = ProteinDataset(test_df, protein_columns, transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    # Initialize Model
    model = CNNModel(num_outputs=len(protein_columns)).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.AdamW(model.parameters(), lr=0.0001, weight_decay=1e-4)
    
    # Train Model
    for epoch in range(10):  # 10 epochs for each fold
        model.train()
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    # Evaluate Model
    model.eval()
    predictions, true_values = [], []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            predictions.append(outputs.cpu().numpy())
            true_values.append(labels.cpu().numpy())

    predictions = np.concatenate(predictions, axis=0)
    true_values = np.concatenate(true_values, axis=0)

    for i, protein in enumerate(protein_columns):
        rmse = np.sqrt(mean_squared_error(true_values[:, i], predictions[:, i]))
        pearson_corr, _ = pearsonr(true_values[:, i], predictions[:, i])
        spearman_corr, _ = spearmanr(true_values[:, i], predictions[:, i])
        r2 = r2_score(true_values[:, i], predictions[:, i])

        metrics_results[protein]["RMSE"].append(rmse)
        metrics_results[protein]["Pearson"].append(pearson_corr)
        metrics_results[protein]["Spearman"].append(spearman_corr)
        metrics_results[protein]["R2"].append(r2)

# Compute Averages
df_metrics = pd.DataFrame({protein: {metric: np.mean(values) for metric, values in results.items()} for protein, results in metrics_results.items()}).T
num_high_spearman = sum(df_metrics["Spearman"] > 0.7)

print("Performance Metrics:")
print(df_metrics)
print(f"Number of proteins with Spearman > 0.7: {num_high_spearman}")


Dataset Columns: Index(['Unnamed: 0', 'VisSpot', 'Location_Center_Y', 'Location_Center_X',
       'SMAa', 'CD11b', 'CD44', 'CD31', 'CDK4', 'YKL40', 'CD11c', 'HIF1a',
       'CD24', 'TMEM119', 'OLIG2', 'GFAP', 'VISTA', 'IBA1', 'CD206', 'PTEN',
       'NESTIN', 'TCIRG1', 'CD74', 'MET', 'P2RY12', 'CD163', 'S100B', 'cMYC',
       'pERK', 'EGFR', 'SOX2', 'HLADR', 'PDGFRa', 'MCT4', 'DNA1', 'DNA3',
       'MHCI', 'CD68', 'CD14', 'KI67', 'CD16', 'SOX10', 'id'],
      dtype='object')
   Unnamed: 0                VisSpot  Location_Center_Y  Location_Center_X  \
0           0  AAACAAGTATCTCCCA-1-A1         142.183168         945.889802   
1           1  AAACAAGTATCTCCCA-1-C1         625.877370         306.087125   
2           2  AAACAATCTACTAGCA-1-A1         604.854953         104.873577   
3           3  AAACAATCTACTAGCA-1-C1         423.898653         493.267222   
4           4  AAACACCAATAACTGC-1-A1          14.059328         192.251135   

       SMAa     CD11b      CD44      CD31      CDK4